# Make tables: calculate p-values and differential expressions for each cancer 

Create a dataframe with p-value results from t-tests for all proteins (trans proteomics when KRAS has missense mutations compared to KRAS wildtype). The dataframe also includes the change in medians between deletions and wildtype. Prepare these tables for further analysis by creating csv files.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u
import plot_utils as p

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Returns a dataframe with proteomics and mutation type

# all_prot: list of trans genes

def all_prot_format_df(cancer_object, all_prot, gene_in = 'KRAS', utils = u):
    mut_type = cancer_object.get_genotype_all_vars(gene_in)
    mut_type['Mutation'].where(mut_type['Mutation'] != 'Missense_Mutation', 'Missense', inplace = True) # replace when false
    mut_type['Mutation'].where(mut_type['Mutation'] != 'nonsynonymous SNV', 'Missense', inplace = True)

    # Keep only tumor samples from proteomics
    prot_and_mutations = cancer_object.join_omics_to_mutations(
        mutations_genes = [gene_in], omics_df_name = 'proteomics', omics_genes = all_prot, tissue_type = 'tumor')
    # Reduce a multiindex 
    if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
        prot_and_mutations = utils.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
        prot_and_mutations = rename_duplicate_cols(prot_and_mutations)
    
    # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
    mut_type = mut_type[['Mutation']]
    prot_df = prot_and_mutations.iloc[:,:-4] # Keep only proteomics
    merged = prot_df.join(mut_type)

    # Keep only Wildtype and deletion
    compare = ['Wildtype_Tumor','Missense']
    get = merged['Mutation'].isin(compare)
    mut_wt = merged[get]
    
    mut_wt = mut_wt.rename(columns={col: re.sub(r'_proteomics', '', col) for col in mut_wt.columns.tolist()})
    return mut_wt


In [4]:
'''
Params
cancer_object: Object. Variable for the loaded cancer data set.
cancer_name: Str. name to add to the created dataframe.
del_wt_df: DataFrame. Only samples with deletions and wildtype for PTEN. 
all_prot_list: List. All proteins in proteomics data frame. 

Returns a dataframe with the difference in medians between proteomics with PTEN wt and del (wt - del). 
'''

def get_change_in_medians_df(cancer_object, cancer_name, del_wt_df, all_prot_list):
    d = del_wt_df[del_wt_df.Mutation == "Missense"]
    wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    del_med = d.median()
    wt_med = wt.median()

    med_dict = {}
    
    # Correlation: + is mutant up compared to wt, - is mutant down
    for prot in all_prot_list:
        if prot in del_med.index and prot in wt_med.index:
            dif = del_med[prot] - wt_med[prot]
            med_dict[prot] = dif
        
    df = pd.DataFrame.from_dict(med_dict, orient='index', columns=[cancer_name+'_Median'])
    df = df.reset_index().rename(columns={'index':'Proteomics'})
    
    return df
        

In [5]:
# Adds '_i' to duplicate col names where i is a number (increases with each duplicate)

def rename_duplicate_cols(df):
    cols = pd.Series(df.columns[:])

    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

    # rename the columns with the cols list.
    df.columns=cols
    return df

#  Step 1: Create data frames with p-values and differential expressions

Each cancer needs a data frame containing only samples that have PTEN cnv deletions and PTEN wildtype with trans proteomics. Use wrap_ttest to run t-tests for all genes in the proteomic data frame. Use get_change_in_medians_df to create the data frame with differential expression values. Merge both data frames. 

Load in cancer data sets from cptac. 

In [6]:
en = cptac.Endometrial()
l = cptac.Luad()
col = cptac.Colon()

# Calculate p-values and differential expressions

In [7]:
cancer_objects = {'Endo':en, 'Colon':col, 'Luad':l}

In [8]:
import warnings
warnings.filterwarnings('ignore')

gene = 'KRAS'
merged_dfs = {}
for cancer in cancer_objects:
    prot = cancer_objects[cancer].get_proteomics()
    prot = prot.drop(columns = 'KRAS') # cis effect
    
    # Reduce a multiindex 
    if isinstance(prot.keys(), pd.core.indexes.multi.MultiIndex):
        prot = u.reduce_multiindex(prot, levels_to_drop = 1)
    prot_list = list(prot.columns)
    
    # Format df for t-test
    del_wt = all_prot_format_df(cancer_objects[cancer], prot_list)
    
    # Rename duplicate columns (isoforms)
    col_names = pd.Series(del_wt.columns[:])
    for dup in col_names[col_names.duplicated()].unique(): 
        col_names[col_names[col_names == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(col_names == dup))]
    del_wt.columns = col_names # rename the columns with the cols list
    
    # T-test
    cols_in = list(del_wt.columns[:-2])

    # Get all pvals
    min_num = 5
    all_pval = u.wrap_ttest(del_wt, 'Mutation', cols_in, return_all = True, correction_method = 'fdr_bh', mincount = min_num)
    all_pval = all_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': cancer+'_P_Value'})
    
    # Get difference in medians
    delta_median_df = get_change_in_medians_df(cancer_objects[cancer], cancer, del_wt, cols_in)

    # Merge pval_df and delta_median_df
    pval_medians_df = all_pval.merge(delta_median_df, on='Proteomics', how='outer')
    pval_medians_df = pval_medians_df.replace(to_replace = r'_proteomics', value = '', regex = True)
    merged_dfs[cancer] = pval_medians_df # testing purposes
    
    # Create csv
    pval_medians_df.to_csv('csv/Single_Cancer/'+cancer+'_pval_medians.csv', index=False)
    print(pval_medians_df.head(), '\n')

  Proteomics  Endo_P_Value  Endo_Median
0      SPRY1      0.001049      0.86320
1  TNFRSF12A      0.032982      0.55361
2    ZFYVE19      0.032982      0.28900
3     GPRC5A      0.032982      1.14670
4      SPRY4      0.035012      0.61115 

  Proteomics  Colon_P_Value  Colon_Median
0     TSPAN8       0.299110       0.55150
1       LY6E       0.374275      -1.01600
2    PLEKHA5       0.452662       0.20575
3       CMAS       0.452662       0.31370
4      HPGDS       0.452662       0.58350 

  Proteomics  Luad_P_Value  Luad_Median
0      DOCK5      0.001950      1.09795
1      TUFT1      0.002112      1.60680
2         F3      0.003637      2.78775
3     CYP4X1      0.004582     -2.68340
4       MPP6      0.008483     -1.22155 



# Get number of significant genes in each cancer

In [9]:
# Using corrected pvals from wrap_ttest, each cancer's cutoff is 0.05
all_sig = []

for cancer in merged_dfs:
    df = merged_dfs[cancer]
    sig_df = df.loc[df[cancer+'_P_Value'] < 0.05]
    print(cancer, 'sig comparisons:', len(sig_df))
    sig_list = list(sig_df['Proteomics'])
    all_sig.append(sig_list)
        

flat_list = [item for sublist in all_sig for item in sublist] #change list of lists, to just one list
sig = list(set(flat_list)) # keep only one if gene in multiple cancer sig lists
print('\nNumber of significant comparisons in at least 1 cancer:', len(sig))

Endo sig comparisons: 6
Colon sig comparisons: 0
Luad sig comparisons: 23

Number of significant comparisons in at least 1 cancer: 29


In [10]:
# check

In [11]:
# check
lu = merged_dfs['Luad']
len(lu.Proteomics.unique())
sl = lu.loc[lu['Luad_P_Value'] < 0.05]
list(sl.Proteomics)
lu.loc[lu['Proteomics'] == 'PIK3R1_1'] #ITGAL_isoform_1, PIK3R1_isoform_1, RABGAP1L_isoform_1

,Proteomics,Luad_P_Value,Luad_Median
6908,PIK3R1_1,0.820346,-0.12775


In [12]:
# check
gene = 'PIK3R1'
pr = all_prot_format_df(l, all_prot = [gene])

col_names = pd.Series(pr.columns)

for dup in col_names[col_names.duplicated()].unique(): 
    col_names[col_names[col_names == dup].index.values.tolist()] = [dup + '_isoform_' + str(i) if i != 0 else dup for i in range(sum(col_names == dup))]
pr.columns = col_names # rename the columns with the cols list

pr = pr[[gene+'_1', 'Mutation']]
delt = pr.loc[pr['Mutation'] == 'Missense']
wt = pr.loc[pr['Mutation'] != 'Missense']

dm = float(delt.median())
wtm = float(wt.median())


print(dm, '-', wtm, '=', dm - wtm)



-0.33035000000000003 - -0.2026 = -0.12775000000000003


In [13]:
# check

# Get a list of significant genes in at least one cancer

In [14]:
s = pd.Series(sig)
print('Number of significant genes in at least one cancer:', len(s))
s.to_csv('csv/list_sig_one_cancer.csv', index=False)

Number of significant genes in at least one cancer: 29


# Get a list of significant genes in multiple cancers

In [15]:
# sig in multiple cancers
mult = list(set(i for i in flat_list if flat_list.count(i) > 1)) # Keep genes sig in more than 1 cancer
print('Number of significant genes in mult cancers:', len(mult))

Number of significant genes in mult cancers: 0


In [16]:
m = pd.Series(mult)
m = m.replace(to_replace = r'_proteomics$', value = '', regex = True)
#m.to_csv('csv/list_sig_multiple_cancers.csv', index=False)